In [2]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
base_model = VGG16(input_shape=(224,224,3), include_top=False, weights="imagenet")
head = base_model.output
head = tf.keras.layers.GlobalAveragePooling2D()(head)
head = tf.keras.layers.Dropout(rate=0.2)(head)
head = tf.keras.layers.Dense(2,activation="softmax")(head)
model = tf.keras.models.Model(inputs=base_model.input, outputs=head)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
base_dir = "TinyML/vw_coco2014_96"

In [16]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   rotation_range=10,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   zoom_range=.1,
                                   horizontal_flip=True,
                                   preprocessing_function=preprocess_input)

In [17]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 validation_split=0.1)

In [18]:
train_generator = train_datagen.flow_from_directory(base_dir,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    subset='training,
                                                    shuffle=True,
                                                    seed=1968,
                                                    color_mode='rgb'
                                                    )

Found 109619 images belonging to 2 classes.


In [19]:
val_generator = val_datagen.flow_from_directory(base_dir,
                                                target_size=(224,224),
                                                batch_size=32,
                                                subset='validation',
                                                shuffle=True,
                                                seed=1968,
                                                color_mode='rgb'
                                                )

Found 10961 images belonging to 2 classes.


In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
checkpoint_path = "vgg16/checkpoint"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_best_only=True)

In [25]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=val_generator,
                    validation_steps=len(val_generator),
                    callbacks = [cp_callback],
                    verbose=1
                   )

Epoch 1/10
 538/3426 [===>..........................] - ETA: 21:00:35 - loss: 1.6303 - accuracy: 0.5067

KeyboardInterrupt: 

In [ ]:
test_accuracy = model.evaluate_generator(val_generator
                                         steps=len(val_generator))